In [28]:
import pandas as pd
import numpy as np
from scipy import stats
from utils import generate_data
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [7]:
data = generate_data()

In [52]:
data.x_df.head()

,intercept,days_since_start,days_since_start_squared,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,seasonality_cos,seasonality_sin
0,1.0,-1.729679,2.228932,0,0,0,0,0,0,1,0.999852,0.017213
1,1.0,-1.726517,2.216707,1,0,0,0,0,0,0,0.999407,0.034422
2,1.0,-1.723355,2.204505,0,1,0,0,0,0,0,0.998667,0.051620
3,1.0,-1.720193,2.192325,0,0,1,0,0,0,0,0.997630,0.068802
4,1.0,-1.717031,2.180168,0,0,0,1,0,0,0,0.996298,0.085965


In [900]:
gpois_mdl = sm.GeneralizedPoisson(data.y, data.x_df)

In [901]:
gpois_res = gpois_mdl.fit()

Optimization terminated successfully.
         Current function value: 5.401221
         Iterations: 15
         Function evaluations: 17
         Gradient evaluations: 17


In [902]:
print(gpois_res.summary2())

                              Results: GeneralizedPoisson
Model:                     GeneralizedPoisson       Pseudo R-squared:       0.087      
Dependent Variable:        y                        AIC:                    11852.6740 
Date:                      2022-03-21 09:10         BIC:                    11912.6561 
No. Observations:          1095                     Log-Likelihood:         -5914.3    
Df Model:                  10                       LL-Null:                -6479.3    
Df Residuals:              1084                     LLR p-value:            1.8807e-236
Converged:                 1.0000                   Scale:                  1.0000     
---------------------------------------------------------------------------------------
                          Coef.    Std.Err.     z     P>|z|      [0.025       0.975]   
---------------------------------------------------------------------------------------
intercept                 5.0598 791984.6502   0.0000 1.0000 -

## Extend Generic Likelihood Model

In [956]:
def _ll_latentnorm(y, X, beta, alph):
    mu = (np.dot(X, beta)) # Should we exponentiate this??
    sigma = np.exp(np.dot(X, alph))
    Phi_bar = stats.norm(mu, sigma).cdf(np.log1p(y))
    Phi_underbar = stats.norm(mu, sigma).cdf(np.log(y))
    ll = np.log(Phi_bar - Phi_underbar)
    #print(Phi_bar.sum())
    #print(Phi_underbar.sum())
    #print(ll.sum())
    return ll

def _gradutils(y, X, beta, alph):
    mu = (np.dot(X, beta)) # Should we exponentiate this??
    sigma = np.exp(np.dot(X, alph))

    z_bar = (np.log1p(y) - mu) / sigma 
    z_underbar = (np.log(y) - mu) / sigma

    Phi_bar = stats.norm.cdf(z_bar)
    Phi_underbar = stats.norm.cdf(z_underbar)
    Phi  = Phi_bar - Phi_underbar

    phi_bar = stats.norm.pdf(z_bar)
    phi_underbar = stats.norm.pdf(z_underbar)
    phi = phi_bar - phi_underbar
    
    kappa_0 = phi / Phi
    kappa_1 = (z_bar * phi_bar - z_underbar * phi_underbar) / Phi
    kappa_2 = (z_bar**2 * phi_bar - z_underbar**2 * phi_underbar) / Phi
    kappa_3 = (z_bar**3 * phi_bar - z_underbar**3 * phi_underbar) / Phi
    
    return kappa_0, kappa_1, kappa_2, kappa_3, mu, sigma
    

In [963]:
class MyLatentNormal(GenericLikelihoodModel):
    def __init__(self, endog, exog, **kwds):
        super(MyLatentNormal, self).__init__(endog, exog, **kwds)
        self.nparams = 24
        
        
    def nloglikeobs(self, params):
        exog = self.exog
        endog = self.endog
        beta = params[:12] #first 12 are for mu
        alph = params[12:] #last 12 are for sigma
        ll = _ll_latentnorm(endog, exog, beta, alph)
        return -ll
    
    def score(self, params):
        y = self.endog
        X = self.exog
        beta = params[:12] #first 12 are for mu
        alph = params[12:] #last 12 are for sigma
        
        kappa_0, kappa_1, kappa_2, kappa_3, mu, sigma = _gradutils(y, X, beta, alph)
        
        grad_beta = -(kappa_0 / sigma) @ X
        grad_alph = -kappa_1 @ X
        
        return np.append(grad_beta, grad_alph)
    
    def hessian(self, params):
        y = self.endog
        X = self.exog
        beta = params[:12] #first 12 are for mu
        alph = params[12:] #last 12 are for sigma
        
        kappa_0, kappa_1, kappa_2, kappa_3, mu, sigma = _gradutils(y, X, beta, alph)
        
        #xx = np.einsum('ij,ij-> i', X, X) # row-wise dot product?
        H_beta = - X.T @ np.diag((kappa_0**2 + kappa_1) / sigma**2) @ X
        H_alph = - X.T @ np.diag(kappa_1 * (kappa_1 - 1) + kappa_3) @ X
        H_beta_alph = - X.T @ np.diag((kappa_2 + kappa_0*(kappa_1 - 1)) / sigma) @ X
        H_all = np.block([[H_beta, H_beta_alph], [H_beta_alph, H_alph]]) # 24 x 24

        return H_all
        
    
    def fit(self, start_params=None, maxiter=10000, maxfun=5000, **kwds):
        if start_params == None:
            # Reasonable starting values
            start_params = 0 * np.ones(self.nparams)
            # intercepts
            start_params[0] = np.log(np.mean(self.endog)) # beta
            #start_params[12] = 0
            #start_params[0] = np.floor(self.endog.mean())
        return super(MyLatentNormal, self).fit(start_params=start_params,
                                     maxiter=maxiter, maxfun=maxfun,
                                     **kwds)

In [964]:
names = list(data.x_df)
names_alpha = [s + "_alpha" for s in names]

In [965]:
mod = MyLatentNormal(data.y, data.x_df, extra_params_names=names_alpha)

In [968]:
mod_res = mod.fit(method="ncg", maxiter = 10000) 
# getting division by zero erros for newton (always), cg, ncg (depending on starting value). Hessian warning bfgs --> no std errors, hessian inv problem
# problem is when Phi_bar - Phi_ub = 0

Optimization terminated successfully.
         Current function value: 5.320359
         Iterations: 9
         Function evaluations: 24
         Gradient evaluations: 19
         Hessian evaluations: 9


<ipython-input-956-d734442e9c04>:6: RuntimeWarning: divide by zero encountered in log
  ll = np.log(Phi_bar - Phi_underbar)
<ipython-input-956-d734442e9c04>:27: RuntimeWarning: invalid value encountered in true_divide
  kappa_0 = phi / Phi
<ipython-input-956-d734442e9c04>:28: RuntimeWarning: invalid value encountered in true_divide
  kappa_1 = (z_bar * phi_bar - z_underbar * phi_underbar) / Phi
<ipython-input-956-d734442e9c04>:29: RuntimeWarning: invalid value encountered in true_divide
  kappa_2 = (z_bar**2 * phi_bar - z_underbar**2 * phi_underbar) / Phi
<ipython-input-956-d734442e9c04>:30: RuntimeWarning: invalid value encountered in true_divide
  kappa_3 = (z_bar**3 * phi_bar - z_underbar**3 * phi_underbar) / Phi
<ipython-input-956-d734442e9c04>:6: RuntimeWarning: divide by zero encountered in log
  ll = np.log(Phi_bar - Phi_underbar)
<ipython-input-956-d734442e9c04>:6: RuntimeWarning: divide by zero encountered in log
  ll = np.log(Phi_bar - Phi_underbar)
<ipython-input-956-d734442

In [971]:
print(mod_res.summary())
print(f"True Beta: {data.beta}")
print(f"True Alpha: {data.alpha}")

                            MyLatentNormal Results                            
Dep. Variable:                      y   Log-Likelihood:                -5825.8
Model:                 MyLatentNormal   AIC:                         1.167e+04
Method:            Maximum Likelihood   BIC:                         1.173e+04
Date:                Mon, 21 Mar 2022                                         
Time:                        09:18:59                                         
No. Observations:                1095                                         
Df Residuals:                    1084                                         
Df Model:                          10                                         
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
intercept                          4.9940   4.94e+05   1.01e-05      1.000   -9.69e+05    9

In [972]:
y = data.y
np.log(y.mean())/2

2.5742195648884167

In [973]:
mod.score(mod_res.params)

array([-4.18464013e-05,  4.64606127e-06,  1.15632356e-05,  2.82659107e-04,
       -2.54589346e-04, -1.86152633e-04,  3.13757444e-05, -2.27687482e-04,
       -1.19417669e-04,  4.31965877e-04,  8.39994746e-06,  4.69422783e-05,
        3.34217410e-04, -5.91569752e-05, -9.13337615e-05,  2.94774929e-05,
        4.33653352e-05,  1.01400681e-04,  2.76731581e-05,  4.65410371e-05,
        6.60914462e-05,  1.96682588e-05,  1.46071543e-04, -1.95687996e-04])

In [974]:
np.linalg.inv(-1 * mod.hessian(mod_res.params)) # test

array([[-8.81428846e+23, -4.80873939e+08,  2.10187774e+08,
         8.81428846e+23,  8.81428846e+23,  8.81428846e+23,
         8.81428846e+23,  8.81428846e+23,  8.81428846e+23,
         8.81428846e+23, -5.75811422e+09, -2.75065331e+09,
         1.21319306e+26, -1.89371518e+10,  2.01769622e+10,
        -1.21319306e+26, -1.21319306e+26, -1.21319306e+26,
        -1.21319306e+26, -1.21319306e+26, -1.21319306e+26,
        -1.21319306e+26, -1.86044529e+10, -2.22379159e+10],
       [-1.03109198e+09,  1.18063979e-04, -2.50816295e-05,
         1.05455175e+09,  1.03109198e+09,  1.29969039e+09,
         1.20333913e+09,  1.03109198e+09,  1.08677269e+09,
         1.32530459e+09, -8.37667364e-06,  3.40301893e-05,
         8.33765827e+10, -2.12300851e-05,  1.88961363e-05,
        -1.30372592e+11, -1.00815158e+11, -1.30372592e+11,
        -1.30372592e+11, -1.30372592e+11, -8.33765827e+10,
        -1.73967828e+11, -3.87468161e-05, -1.67024346e-05],
       [-5.10125064e+08, -2.56707047e-05,  1.16942420e

In [975]:
bs = mod_res.bootstrap(nrep=10, method="ncg", store = 0, disp = 0) # we can bootstrap SE
bs[0] #mean estimates
bs[1] # se

/usr/local/anaconda/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
<ipython-input-956-d734442e9c04>:6: RuntimeWarning: divide by zero encountered in log
  ll = np.log(Phi_bar - Phi_underbar)
<ipython-input-956-d734442e9c04>:27: RuntimeWarning: invalid value encountered in true_divide
  kappa_0 = phi / Phi
<ipython-input-956-d734442e9c04>:28: RuntimeWarning: invalid value encountered in true_divide
  kappa_1 = (z_bar * phi_bar - z_underbar * phi_underbar) / Phi
<ipython-input-956-d734442e9c04>:29: RuntimeWarning: invalid value encountered in true_divide
  kappa_2 = (z_bar**2 * phi_bar - z_underbar**2 * phi_underbar) / Phi
<ipython-input-956-d734442e9c04>:30: RuntimeWarning: invalid value encountered in true_divide
  kappa_3 = (z_bar**3 * phi_bar - z_underbar**3 * phi_underbar) / Phi
<ipython-input-956-d734442e9c04>:6: RuntimeWarn